# 🧬 Project Chimera — Phase 2: TFT Training Pipeline

**Temporal Fusion Transformer** for multi-asset 5-day return forecasting.

| Setting | Value |
|---|---|
| Platform | Kaggle (GPU T4 × 2) |
| Lookback | 60 calendar days |
| Horizon | 5 days (`Target_Return_5D`) |
| Model | TFT via `pytorch-forecasting` |
| Precision | FP16-mixed (halves VRAM) |

In [ ]:
# =============================================================================
# Cell 1A — The Installer (Run ONCE, then Restart Session)
# =============================================================================
# ROOT CAUSE: Kaggle uses Python 3.12, but pytorch-forecasting<=1.0.0
# requires Python <3.11.  We need pytorch-forecasting>=1.1.0, which ships
# with the unified "lightning" 2.x package (not the old "pytorch-lightning").

!pip install -q "pytorch-forecasting>=1.1.0"

# Verify what actually got installed
!pip list | grep -iE "forecasting|lightning"

print()
print("=" * 60)
print("✅ DONE. Now click 'Restart Session' in the Run menu.")
print("   After restart, SKIP this cell and run Cell 1B below.")
print("=" * 60)

In [1]:
# =============================================================================
# Cell 1B — The Loader (Run AFTER Restart)
# =============================================================================
# pytorch-forecasting >=1.1 uses the unified "lightning" package.
# Import path is now: lightning.pytorch  (not the old "pytorch-lightning").

try:
    import sys
    import warnings
    warnings.filterwarnings("ignore")

    from pathlib import Path

    import numpy as np
    import pandas as pd
    import torch
    import lightning.pytorch as pl          # ← unified Lightning 2.x

    from pytorch_forecasting import (
        TimeSeriesDataSet,
        TemporalFusionTransformer,
        GroupNormalizer,
    )
    from pytorch_forecasting.metrics import QuantileLoss
    from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor

    # Reproducibility
    pl.seed_everything(42, workers=True)
    NUM_WORKERS: int = 2

    print("✅ Libraries Loaded! Ready to train.")
    print(f"   Python    : {sys.version.split()[0]}")
    print(f"   Torch     : {torch.__version__}")
    print(f"   Lightning : {pl.__version__}")
    print(f"   GPU       : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

except Exception as e:
    print("❌ Error: Did you run Cell 1A and Restart the Session?")
    print(f"   Details: {e}")
    raise

Seed set to 42


✅ Libraries Loaded! Ready to train.
   Python    : 3.12.12
   Torch     : 2.8.0+cu126
   Lightning : 2.6.1
   GPU       : Tesla T4


In [4]:
# =============================================================================
# Cell 2 — Data Loading & Preprocessing
# =============================================================================
# Flexible path: try Kaggle input dir first, then local fallback.
KAGGLE_PATH = Path("/kaggle/input/datasets/wangdylanlorrenzo/chimera-alpha-engine-aligned-market-data-v1/chimera_data_v1.csv")
LOCAL_PATH  = Path("../data/processed/chimera_data_v1.csv")

csv_path = KAGGLE_PATH if KAGGLE_PATH.exists() else LOCAL_PATH
print(f"Loading data from: {csv_path.resolve()}")
raw_wide = pd.read_csv(csv_path, index_col=0, parse_dates=True)
raw_wide.index.name = "Date"

print(f"Wide shape: {raw_wide.shape}")
print(f"Date range: {raw_wide.index.min().date()} → {raw_wide.index.max().date()}")

# ---------------------------------------------------------------------------
# Melt wide format (TICKER_Feature) → long format required by TFT
# ---------------------------------------------------------------------------
# Columns follow the pattern: "TICKER_Feature"
# Parse every column into (ticker, feature) pairs.
FEATURES_PER_TICKER = [
    "Close", "High", "Low", "Open", "Volume",
    "Trend_Spread", "Stoch_K", "Stoch_D", "Is_Oversold",
    "RVOL", "Log_Returns", "Target_Return_5D",
]

records: list[pd.DataFrame] = []
tickers_found: list[str] = []

# Derive unique tickers from columns
col_set = set(raw_wide.columns)
for col in sorted(raw_wide.columns):
    # Find the feature suffix; the ticker is everything before it
    for feat in FEATURES_PER_TICKER:
        if col.endswith(f"_{feat}"):
            ticker = col[: -(len(feat) + 1)]  # strip "_Feature"
            if ticker not in tickers_found:
                # Verify ALL features exist for this ticker
                expected = [f"{ticker}_{f}" for f in FEATURES_PER_TICKER]
                if all(e in col_set for e in expected):
                    tickers_found.append(ticker)
            break

print(f"Tickers detected ({len(tickers_found)}): {tickers_found}")

for ticker in tickers_found:
    sub = raw_wide[[f"{ticker}_{f}" for f in FEATURES_PER_TICKER]].copy()
    sub.columns = FEATURES_PER_TICKER  # strip ticker prefix
    sub["Ticker"] = ticker
    sub["Date"] = sub.index if isinstance(sub.index, pd.DatetimeIndex) else pd.to_datetime(sub.index)
    records.append(sub)

df = pd.concat(records, ignore_index=True)

# ---------------------------------------------------------------------------
# Asset class label (static categorical for TFT)
# ---------------------------------------------------------------------------
CRYPTO_TICKERS: set[str] = {"BTC-USD", "ETH-USD", "SOL-USD"}
df["Asset_Class"] = df["Ticker"].apply(
    lambda t: "CRYPTO" if t in CRYPTO_TICKERS else "FINTECH"
)

# ---------------------------------------------------------------------------
# CRITICAL (TFT requirement): time_idx — continuous integer per group
# ---------------------------------------------------------------------------
df.sort_values(["Ticker", "Date"], inplace=True)
df["time_idx"] = df.groupby("Ticker").cumcount()

# Cast categoricals
df["Ticker"]      = df["Ticker"].astype(str)
df["Asset_Class"] = df["Asset_Class"].astype(str)

# Fill any residual NaNs in numeric columns to prevent NaN loss
numeric_cols = df.select_dtypes(include="number").columns
df[numeric_cols] = df[numeric_cols].fillna(0)

df.reset_index(drop=True, inplace=True)

print(f"\nLong-format shape : {df.shape}")
print(f"time_idx range    : {df['time_idx'].min()} → {df['time_idx'].max()}")
print(f"Tickers           : {df['Ticker'].nunique()}")
print(f"Asset classes      : {df['Asset_Class'].unique().tolist()}")
df.head()

Loading data from: /kaggle/input/datasets/wangdylanlorrenzo/chimera-alpha-engine-aligned-market-data-v1/chimera_data_v1.csv
Wide shape: (1649, 120)
Date range: 2021-07-30 → 2026-02-02
Tickers detected (10): ['BTC-USD', 'COIN', 'DX-Y.NYB', 'ETH-USD', 'HOOD', 'MA', 'PYPL', 'SOL-USD', 'V', '^TNX']

Long-format shape : (16490, 16)
time_idx range    : 0 → 1648
Tickers           : 10
Asset classes      : ['CRYPTO', 'FINTECH']


,Close,High,Low,Open,Volume,Trend_Spread,Stoch_K,Stoch_D,Is_Oversold,RVOL,Log_Returns,Target_Return_5D,Ticker,Date,Asset_Class,time_idx
0,42235.546875,42235.546875,38397.355469,40027.484375,3.307278e+10,0.025948,95.263215,90.712255,0,1.299297,0.054172,-0.058909,BTC-USD,2021-07-30,CRYPTO,0
1,41626.195312,42231.449219,41110.832031,42196.304688,2.580285e+10,0.029934,96.072158,94.442837,0,1.002478,-0.014533,-0.018177,BTC-USD,2021-07-31,CRYPTO,1
2,39974.894531,42541.679688,39540.941406,41460.843750,2.668844e+10,0.031547,91.931081,94.422151,0,1.032139,-0.040478,0.071085,BTC-USD,2021-08-01,CRYPTO,2
3,39201.945312,40419.179688,38746.347656,39907.261719,2.559527e+10,0.031026,83.485075,90.496105,0,0.977623,-0.019525,0.136571,BTC-USD,2021-08-02,CRYPTO,3
4,38152.980469,39750.031250,37782.050781,39178.402344,2.618983e+10,0.029121,73.823057,83.079738,0,0.991221,-0.027122,0.147961,BTC-USD,2021-08-03,CRYPTO,4


In [6]:
# =============================================================================
# Cell 3 — TimeSeriesDataSet Configuration (The Architecture)
# =============================================================================
# The TFT requires a structured TimeSeriesDataSet that declares the role of
# every column: static vs. time-varying, known vs. unknown, real vs. categorical.

# Training cutoff: everything except the last 60 time steps per group
max_time_idx: int = df["time_idx"].max()
training_cutoff: int = max_time_idx - 60  # last 60 days → validation

print(f"Max time_idx     : {max_time_idx}")
print(f"Training cutoff  : {training_cutoff}")
print(f"Train rows (est) : {(df['time_idx'] <= training_cutoff).sum()}")
print(f"Val rows (est)   : {(df['time_idx'] >  training_cutoff).sum()}")

# --- Time-varying unknown reals ------------------------------------------
# These are the Phase-1 engineered features the model will learn from.
# NOTE: BB_Width was specified in the plan but is not present in the Phase-1
# feature set.  We use the features that actually exist in the CSV.
TIME_VARYING_UNKNOWN_REALS: list[str] = [
    "Close",
    "Log_Returns",
    "Trend_Spread",
    "Stoch_K",
    "Stoch_D",
    "RVOL",
]

training = TimeSeriesDataSet(
    df[df["time_idx"] <= training_cutoff],
    time_idx="time_idx",
    target="Target_Return_5D",
    group_ids=["Ticker"],
    min_encoder_length=60,          # fixed 60-day lookback
    max_encoder_length=60,
    min_prediction_length=5,        # 5-day forecast horizon
    max_prediction_length=5,
    static_categoricals=["Asset_Class", "Ticker"],
    time_varying_unknown_reals=TIME_VARYING_UNKNOWN_REALS,
    target_normalizer=GroupNormalizer(
        groups=["Ticker"],
        transformation="softplus",   # ensures positive scale parameters
    ),
    add_relative_time_idx=True,      # relative position inside the window
    add_target_scales=True,          # passes (center, scale) to the model
    add_encoder_length=True,         # lets model know how long the input is
)

print(f"\n✅ TimeSeriesDataSet created — {len(training)} samples")
print(f"   Encoder length : {training.max_encoder_length}")
print(f"   Prediction len : {training.max_prediction_length}")

Max time_idx     : 1648
Training cutoff  : 1588
Train rows (est) : 15890
Val rows (est)   : 600

✅ TimeSeriesDataSet created — 15250 samples
   Encoder length : 60
   Prediction len : 5


In [7]:
# =============================================================================
# Cell 4 — Dataloaders
# =============================================================================
# Memory-safety: batch_size=64 keeps peak VRAM well within T4's 16 GB.
BATCH_SIZE: int = 64

# Validation set: the LAST 60 days held out via training_cutoff
validation = TimeSeriesDataSet.from_dataset(
    training,
    df,                              # full dataframe (includes val period)
    predict=True,                    # marks this as the prediction/val split
    stop_randomization=True,         # deterministic iteration for val
)

train_dataloader = training.to_dataloader(
    train=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    persistent_workers=True,
)

val_dataloader = validation.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE * 2,       # no grads → can afford larger batches
    num_workers=NUM_WORKERS,
    persistent_workers=True,
)

print(f"Train batches : {len(train_dataloader)}")
print(f"Val batches   : {len(val_dataloader)}")
print(f"Batch size    : {BATCH_SIZE} (effective {BATCH_SIZE * 2} via grad accumulation)")

Train batches : 238
Val batches   : 1
Batch size    : 64 (effective 128 via grad accumulation)


In [ ]:
# =============================================================================
# Cell 5 — TFT Model Initialization (Fix: Disable Interpretation Logging)
# =============================================================================
# The Temporal Fusion Transformer combines:
#   • Variable Selection Networks  → automatic feature importance
#   • Gated Residual Networks      → skip connections for depth
#   • Multi-Head Attention         → long-range temporal patterns
#   • LSTM encoder/decoder         → sequential dependencies
#
# QuantileLoss produces a distribution (P10, P25, P50, P75, P90, P95, P99)
# instead of a single point estimate, giving us actionable confidence bands.

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=64,                  # balance between capacity and VRAM
    lstm_layers=2,
    dropout=0.1,
    output_size=7,                   # 7 quantiles for QuantileLoss
    loss=QuantileLoss(),
    attention_head_size=4,
    reduce_on_plateau_patience=4,    # LR scheduler patience

    # CRITICAL FIX: Disable auto-logging of interpretation to prevent KeyError.
    # pytorch-forecasting's interpretation hook looks for an 'interpretation'
    # key in the model output dict, which isn't always present depending on
    # the version. Setting these to 0 disables the hook entirely.
    # We can still call tft.interpret_output() manually after training.
    log_interval=0,
    log_val_interval=0,
)

num_params = sum(p.numel() for p in tft.parameters() if p.requires_grad)
print(f"✅ TFT initialised — {num_params:,} trainable parameters")
print(f"   Hidden size     : 64")
print(f"   LSTM layers     : 2")
print(f"   Attention heads : 4")
print(f"   Loss            : QuantileLoss (7 quantiles)")
print(f"   Logging         : Disabled (prevents interpretation KeyError)")

✅ TFT initialised — 293,714 trainable parameters
   Hidden size     : 64
   LSTM layers     : 2
   Attention heads : 4
   Loss            : QuantileLoss (7 quantiles)


In [11]:
# =============================================================================
# Cell 6 — Trainer (Lightning 2.x Configuration)
# =============================================================================
# IMPORTANT: precision="16-mixed" causes RuntimeError in the TFT attention
# mask layer because the mask bias (-1e9) overflows FP16's max (~65504).
# We use precision="32" instead. With hidden_size=64 and batch_size=64,
# the model fits comfortably in T4's 16 GB VRAM even at full precision.
#
# Other OOM-prevention strategies:
#   • accumulate_grad_batches=2  → effective batch = 128
#   • gradient_clip_val=0.1      → tames spiky financial gradients

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=True,
    mode="min",
)

lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    precision=32,                     # FP32 — avoids TFT attention mask overflow
    accumulate_grad_batches=2,
    max_epochs=20,
    gradient_clip_val=0.1,
    callbacks=[early_stop, lr_monitor],
    enable_progress_bar=True,
    log_every_n_steps=10,
)

print("✅ Trainer ready (Lightning 2.x — FP32)")
print(f"   Precision         : 32 (FP16 incompatible with TFT attention mask)")
print(f"   Max epochs        : 20")
print(f"   Grad accumulation : 2  (effective batch = {BATCH_SIZE * 2})")
print(f"   Gradient clip     : 0.1")
print(f"   Early stopping    : patience=5 on val_loss")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


✅ Trainer ready (Lightning 2.x — FP32)
   Precision         : 32 (FP16 incompatible with TFT attention mask)
   Max epochs        : 20
   Grad accumulation : 2  (effective batch = 128)
   Gradient clip     : 0.1
   Early stopping    : patience=5 on val_loss


In [ ]:
# =============================================================================
# Cell 7 — Execution & Model Saving
# =============================================================================
# Train the TFT on all data up to the cutoff, validating on the last 60 days.

# WORKAROUND: cuDNN's fused RNN kernel has a bug where it loses training-mode
# state during backward, causing "cudnn RNN backward can only be called in
# training mode". Disabling cuDNN forces PyTorch's native LSTM implementation,
# which is slightly slower but correct.
torch.backends.cudnn.enabled = False

trainer.fit(tft, train_dataloader, val_dataloader)

# Re-enable cuDNN for any downstream GPU ops (predictions, etc.)
torch.backends.cudnn.enabled = True

# --- Load best checkpoint (lowest val_loss) ---------------------------------
best_model_path = trainer.checkpoint_callback.best_model_path
print(f"\n🏆 Best checkpoint: {best_model_path}")

best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# --- Save as a portable .ckpt file for easy download from Kaggle ------------
# IMPORTANT: We copy the FULL Lightning checkpoint (not just state_dict).
# load_from_checkpoint() requires the full checkpoint format which includes
# hyperparameters, optimizer state, and Lightning metadata.
# Saving only state_dict via torch.save() would cause:
#   KeyError: 'pytorch-lightning_version'
import shutil

SAVE_PATH = Path("best_tft_model.ckpt")
shutil.copy2(best_model_path, SAVE_PATH)
print(f"✅ Model saved → {SAVE_PATH.resolve()}  ({SAVE_PATH.stat().st_size / 1e6:.1f} MB)")

# --- Quick sanity check: predict on validation set --------------------------
val_predictions = best_tft.predict(val_dataloader, mode="quantiles")
print(f"\nValidation predictions shape: {val_predictions.shape}")
print(f"Sample quantiles (first batch, first sample):\n{val_predictions[0]}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┓
┃    ┃ Name                               ┃ Type                            ┃ Params ┃ Mode ┃ FLOPs ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━┩
│ 0  │ loss                               │ QuantileLoss                    │      0 │ eval │     0 │
│ 1  │ logging_metrics                    │ ModuleList                      │      0 │ eval │     0 │
│ 2  │ input_embeddings                   │ MultiEmbedding                  │     62 │ eval │     0 │
│ 3  │ prescalers                         │ ModuleDict                      │    160 │ eval │     0 │
│ 4  │ static_variable_selection          │ VariableSelectionNetwork        │  5.6 K │ eval │     0 │
│ 5  │ encoder_variable_selection         │ VariableSelectionNetwork        │ 12.5 K │ eval │     0 │
│ 6  │ decoder_variable_selection         │ VariableSelectionNetwork        │  1.6 K │ eval │     0 │
│ 7  │ static_context_variable_selection  │ GatedResidualNetwork            │ 16.8 K │ eval │     0 │
│ 8  │ static_context_initial_hidden_lstm │ GatedResidualNetwork            │ 16.8 K │ eval │     0 │
│ 9  │ static_context_initial_cell_lstm   │ GatedResidualNetwork            │ 16.8 K │ eval │     0 │
│ 10 │ static_context_enrichment          │ GatedResidualNetwork            │ 16.8 K │ eval │     0 │
│ 11 │ lstm_encoder                       │ LSTM                            │ 66.6 K │ eval │     0 │
│ 12 │ lstm_decoder                       │ LSTM                            │ 66.6 K │ eval │     0 │
│ 13 │ post_lstm_gate_encoder             │ GatedLinearUnit                 │  8.3 K │ eval │     0 │
│ 14 │ post_lstm_add_norm_encoder         │ AddNorm                         │    128 │ eval │     0 │
│ 15 │ static_enrichment                  │ GatedResidualNetwork            │ 20.9 K │ eval │     0 │
│ 16 │ multihead_attn                     │ InterpretableMultiHeadAttention │ 10.4 K │ eval │     0 │
│ 17 │ post_attn_gate_norm                │ GateAddNorm                     │  8.4 K │ eval │     0 │
│ 18 │ pos_wise_ff                        │ GatedResidualNetwork            │ 16.8 K │ eval │     0 │
│ 19 │ pre_output_gate_norm               │ GateAddNorm                     │  8.4 K │ eval │     0 │
│ 20 │ output_layer                       │ Linear                          │    455 │ eval │     0 │
└────┴────────────────────────────────────┴─────────────────────────────────┴────────┴──────┴───────┘

Trainable params: 293 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 293 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 0                                                                                           
Modules in eval mode: 320                                                                                          
Total FLOPs: 0

Output()

KeyError: 'interpretation'